In [ ]:
from serpapi import GoogleSearch

api_key = "Your SerpAPI Key"
your_id = 'Your Google Scholar Author ID'

def getPubList(author_id):
    params = {
      "api_key": api_key,
      "engine": "google_scholar_author",
      "hl": "en",
      "author_id": author_id,
      "num": "100",
      "no_cache": "true"
    }
    
    search = GoogleSearch(params)
    results = search.get_dict()['articles']

    pubList=[]
    for pubs in results:
        if pubs['cited_by']['value'] != None:
            pubList.append([pubs['cited_by']['cites_id'], pubs['title']])
    return pubList



def getCitationPaper(Publication):
    params = {
      "api_key": api_key,
      "engine": "google_scholar",
      "hl": "en",
      "cites": Publication[0],
      "num": "20",
      "no_cache": "true"
    }
    
    search = GoogleSearch(params)
    results = search.get_dict()

    print(results['search_information'])
    numOfCite = results['search_information']['total_results']
    cited_paper_list = []
    
    
    for pubs in results['organic_results']:
        cited_paper_list.append([pubs['title'], Publication[1]])
    
    if numOfCite > 20:
        for start_id in range(20, numOfCite, 20):
            
            params = {
              "api_key": api_key,
              "engine": "google_scholar",
              "hl": "en",
              "cites": Publication[0],
              "no_cache": "true",
              "num": "20",
              "start": str(start_id)
            }
            search = GoogleSearch(params)
            results = search.get_dict()

            print("range:"+str(start_id)+"~"+str(start_id+20))
            for pubs in results['organic_results']:
                cited_paper_list.append([pubs['title'], Publication[1]])
            
    return cited_paper_list

def getCitationNum(author_info):
    params = {
      "engine": "google_scholar_author",
      "author_id": author_info[0],
      "api_key": api_key,
      "no_cache": "true"
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    author_name = results['author']['name']
    url = results['search_metadata']['google_scholar_author_url']
    author_id = results['search_parameters']['author_id']

    if 'cited_by' in results:
        cited_by = results["cited_by"]
        overall_citations = cited_by['table'][0]['citations']['all']
        h_index = cited_by['table'][1]['h_index']['all']
        i10_index = cited_by['table'][2]['i10_index']['all']
        ret = [author_name, overall_citations, h_index, i10_index, author_info[2], author_info[3], author_id, url ]
        return ret
    else:
        return None

def getCitationAuthorId(citation):
    params = {
      "api_key": api_key,
      "engine": "google_scholar",
      "q": citation[0],
      "hl": "en",
      "no_cache": "true"
    }
    
    search = GoogleSearch(params)
    results = search.get_dict()

    output = []
    if 'authors' in results['organic_results'][0]['publication_info']:
        for author in results['organic_results'][0]['publication_info']['authors']:
            output.append([author['author_id'], author['name']] + citation)
    return output

### get Publication List

In [ ]:
result = getPubList(your_id)

# If one publication have several citation Id, split it into several items
result2 = []
for item in result:
    tmp = item[0].split(',')
    for id in tmp:
        result2.append([id, item[1]])

with open('./file1.csv', 'w+', newline='') as f:
    writer = csv.writer(f,delimiter=',')
    for item in result2:
        writer.writerow(item)

### get Citation Paper List

In [ ]:
pubIdList = []
with open('./file1.csv', 'r+') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        pubIdList.append(row)

for i in pubIdList:
    print(i)

In [ ]:
# If the pubIdList is very long, you'd better split it 
# into several seperated list for parallel execution 

# for example

# list1 = pubIdList[0]
# list2 = pubIdList[1:3]
# list3 = pubIdList[3:5]
# list4 = pubIdList[5:7]
# list5 = pubIdList[7:11]
# list6 = pubIdList[11:18]
# list7 = pubIdList[18:35]
# list8 = pubIdList[35:]




In [ ]:
citation_PubList=[] #this line

for pubs in pubIdList:
    citation_PubList2 += (getCitationPaper(pubs))

with open('file2.csv', 'w+', newline='') as f:
    writer = csv.writer(f,delimiter=',')
    for item in citation_PubList: 
        writer.writerow(item)

### get CitationAuthorID List

In [ ]:
citation_PubList=[]
with open('file2.csv', 'r+') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        citation_PubList.append(row)

cnt =0
print("total workload:"+str(len(citation_PubList1[:340])))
for citationPaper in citation_PubList1[:340]:
    if cnt%10==0:
        print("duration:"+str(cnt)+"~"+str(cnt+10))
    authorIdList+= getCitationAuthorId(citationPaper)
    cnt +=1

with open('./terry/file3_list1.csv', 'w+', newline='') as f:
    writer = csv.writer(f,delimiter=',')
    for item in authorIdList:
        writer.writerow(item)

### get CitationNum of Citation Author List

In [ ]:
authorIdList=[]
with open('file3.csv', 'r+') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        authorIdList.append(row)


cnt =0
print("total workload:"+str(len(authorIdList[787:])))
for author in authorIdList[787:]:
    if cnt%10==0:
        print("duration:"+str(cnt)+"~"+str(cnt+10))
    ret_value = getCitationNum(author)
    if ret_value != None:
        CitationNum.append(ret_value)
    cnt+=1

with open('./terry/file4_list1.csv', 'w+', newline='') as f:
    writer = csv.writer(f,delimiter=',')
    for item in CitationNum:
        writer.writerow(item)